In [1]:
import tensorflow as tf
import numpy as np
import random

tf.compat.v1.disable_eager_execution()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
INFO:tensorflow:Disabling eager execution


In [2]:
NUM_CLASSES = 20

In [3]:
class MLP:
    def __init__(self, vocab_size, hidden_size):
        self._vocab_size = vocab_size
        self._hidden_size = hidden_size

    def build_graph(self):
        self._X = tf.compat.v1.placeholder(tf.float32, shape=[None, self._vocab_size])
        self._real_Y = tf.compat.v1.placeholder(tf.int32, shape=[None,])

        weights_1 = tf.compat.v1.get_variable(
            name='weights_input_hidden',
            shape=(self._vocab_size, self._hidden_size),
            initializer=tf.random_normal_initializer(seed=2021),
        )
        biases_1 = tf.compat.v1.get_variable(
            name='biases_input_hidden',
            shape=(self._hidden_size),
            initializer=tf.random_normal_initializer(seed=2021)
        )

        weights_2 = tf.compat.v1.get_variable(
            name='weights_output_hidden',
            shape=(self._hidden_size, NUM_CLASSES),
            initializer=tf.random_normal_initializer(seed=2021),
        )
        biases_2 = tf.compat.v1.get_variable(
            name='biases_output_hidden',
            shape=(NUM_CLASSES),
            initializer=tf.random_normal_initializer(seed=2021)
        )

        hidden = tf.matmul(self._X, weights_1) + biases_1
        hidden = tf.sigmoid(hidden)

        logits = tf.matmul(hidden, weights_2) + biases_2

        labels_one_hot = tf.one_hot(indices=self._real_Y, depth=NUM_CLASSES, dtype=tf.float32)
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels_one_hot, logits=logits)
        loss = tf.reduce_mean(loss)

        probs = tf.nn.softmax(logits)
        predicted_labels = tf.argmax(probs, axis=1)
        predicted_labels = tf.squeeze(predicted_labels)

        return predicted_labels, loss

    def trainer(self, loss, learning_rate):
        with tf.compat.v1.variable_scope(tf.compat.v1.get_variable_scope(),reuse=tf.compat.v1.AUTO_REUSE): 
            train_op = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
        return train_op


In [4]:
class DataReader:
    def __init__(self, data_path, batch_size, vocab_size):
        self._batch_size = batch_size
        with open(data_path) as f:
            d_lines = f.read().splitlines()
        
        self._data = []
        self._labels = []
        for data_id, line in enumerate(d_lines):
            vector = [0.0 for _ in range(vocab_size)]
            features = line.split('<fff>')
            label, doc_id = int(features[0]), int(features[1])
            tokens = features[2].split()

            for token in tokens:
                index, value = int(token.split(':')[0]), float(token.split(':')[1])
                vector[index] = value
            
            self._data.append(vector)
            self._labels.append(label)

        self._data = np.array(self._data)
        self._labels = np.array(self._labels)

        self._num_epoch = 0
        self._batch_id = 0   

    def next_batch(self):
        start = self._batch_id * self._batch_size
        end = start + self._batch_size
        self._batch_id += 1

        if end + self._batch_size > len(self._data):
            end = len(self._data)
            self._num_epoch += 1
            self._batch_id = 0
            indices = list(range(len(self._data)))
            random.seed(2021)
            random.shuffle(indices)
            self._data, self._labels = self._data[indices], self._labels[indices]
        return self._data[start:end], self._labels[start:end]


In [5]:
def load_dataset():
    train_data_reader = DataReader(
        data_path='../datasets/20news-bydate/20news-train-tfidf.txt',
        batch_size=50,
        vocab_size=vocab_size
    )
    test_data_reader = DataReader(
        data_path='../datasets/20news-bydate/20news-test-tfidf.txt',
        batch_size=50,
        vocab_size=vocab_size
    )
    return train_data_reader, test_data_reader

In [6]:
def save_parameters(name, value, epoch):
    filename = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
    if len(value.shape) == 1:
        string_form = ','.join([str(number) for number in value])
    else:
        string_form = '\n'.join([','.join([str(number) for number in value[row]]) for row in range(value.shape[0])])
    
    with open('./saved-paras/' + filename, 'w') as f:
        f.write(string_form)

In [7]:
def restore_parameters(name, epoch):
    filename = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
    with open('./saved-paras/' + filename, 'r') as f:
        lines = f.read().splitlines()
    if len(lines) == 1:
        value = [float(number) for number in lines[0].split(',')]
    else:
        value = [[float(number) for number in lines[row].split(',')] for row in range(len(lines))]
    return value

In [8]:
with open('../datasets/20news-bydate/words_idfs.txt') as f:
    vocab_size = len(f.read().splitlines())

mlp = MLP(
    vocab_size=vocab_size,
    hidden_size=50
)

predicted_labels, loss = mlp.build_graph()
train_op = mlp.trainer(loss=loss, learning_rate=0.1)

train_data_reader, test_data_reader = load_dataset()

with tf.compat.v1.Session() as sess:
    step, MAX_STEP = 0, 20000

    sess.run(tf.compat.v1.global_variables_initializer())
    while step < MAX_STEP:
        train_data, train_labels = train_data_reader.next_batch()
        plabels_eval, loss_eval, _ = sess.run(
            [predicted_labels, loss, train_op],
            feed_dict={
                mlp._X: train_data,
                mlp._real_Y: train_labels
            }
        )
        step += 1
        print('Step: {}. Loss: {}'.format(step, loss_eval))

    trainable_variables = tf.compat.v1.trainable_variables()
    for variable in trainable_variables:
        save_parameters(
            name=variable.name,
            value=variable.eval(),
            epoch=train_data_reader._num_epoch
        )


with tf.compat.v1.Session() as sess:
    epoch = train_data_reader._num_epoch
    trainable_variables = tf.compat.v1.trainable_variables()
    for variable in trainable_variables:
        saved_value = restore_parameters(variable.name, epoch)
        assign_op = variable.assign(saved_value)
        sess.run(assign_op)
    
    num_true_preds = 0
    while True:
        test_data, test_labels = test_data_reader.next_batch()
        test_plabels_eval = sess.run(
            predicted_labels,
            feed_dict={
                mlp._X: test_data,
                mlp._real_Y: test_labels
            }
        )
        matches = np.equal(test_plabels_eval, test_labels)
        num_true_preds += np.sum(matches.astype(float))

        if test_data_reader._batch_id == 0:
            break
    print('Epoch: {}'.format(epoch))
    print('Accuracy on test data: {}'.format(num_true_preds/len(test_data_reader._data)))
    

72139e-08
Step: 19514. Loss: 5.7220425730974966e-08
Step: 19515. Loss: 3.3378452712895523e-07
Step: 19516. Loss: 4.768367389829109e-08
Step: 19517. Loss: 1.506749299551302e-06
Step: 19518. Loss: 3.099441059362107e-08
Step: 19519. Loss: 6.198876434382328e-08
Step: 19520. Loss: 1.7404494201400666e-07
Step: 19521. Loss: 1.192092646817855e-08
Step: 19522. Loss: 1.811978904697753e-07
Step: 19523. Loss: 0.00017130190099123865
Step: 19524. Loss: 1.430511176181426e-08
Step: 19525. Loss: 1.4066682751945336e-07
Step: 19526. Loss: 3.0134581265883753e-06
Step: 19527. Loss: 7.390973166820913e-08
Step: 19528. Loss: 5.316686610967736e-07
Step: 19529. Loss: 2.6009788598457817e-06
Step: 19530. Loss: 3.943071533285547e-06
Step: 19531. Loss: 2.074236391536033e-07
Step: 19532. Loss: 0.16136722266674042
Step: 19533. Loss: 1.0514010000406415e-06
Step: 19534. Loss: 7.629385123664179e-08
Step: 19535. Loss: 1.2397752868764655e-07
Step: 19536. Loss: 4.529947972287118e-08
Step: 19537. Loss: 3.576278118089249e-08